In [1]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M15'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]       = Query[['a','z']]
    Src[['Symbol','TF']] = Query[['symbol','tf']]
    
    Src['Datetime']     = Query['datetime']
    Src['Date']         = Query['datetime'].dt.date
    Src['Time']         = Query['datetime'].dt.time
    
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [5]:
def CALCULATIONS(Src):
    # ================ Helpers ================ #
    def _int(x):
        try:    return int(x)
        except: return x
        
    # ================ Main ================ #
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']    = Calc_by['Open']  .transform('first')
    Calc['Day High']    = Calc_by['High']  .transform('max')
    Calc['Day Low']     = Calc_by['Low']   .transform('min')
    Calc['Day Close']   = Calc_by['Close'] .transform('last')

    Calc['Day HL']      = Calc['Day High']  - Calc['Day Low']
    Calc['Day Chg']     = Calc['Day Close'] - Calc['Day Open']
    Calc['Day Chg Abs'] = Calc['Day Chg'].abs()

    Calc['ID High Rnk'] = Calc_by['High'].rank(method='dense', ascending=0)
    Calc['ID Low Rnk']  = Calc_by['Low'].rank(method='dense', ascending=1)


    WINS_CHG = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6500,                   inf]
    WINS_HL  = [   500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6500, 7000, 7500, 8000, inf]

    for (a,b) in [x for x in pd.Series(WINS_CHG).rolling(2) if (len(x)==2)]:  Calc[f'Day Chg Abs {_int(a)}-{_int(b)}'] = ((a <= Calc['Day Chg Abs']) & (Calc['Day Chg Abs'] < b))  .astype(int)
    for (a,b) in [x for x in pd.Series(WINS_HL).rolling(2)  if (len(x)==2)]:       Calc[f'Day HL {_int(a)}-{_int(b)}'] = ((a <= Calc['Day HL'])      & (Calc['Day HL']      < b))  .astype(int)
    return Calc

Calc = CALCULATIONS(Src)
# Calc.head()

# Stats

In [6]:
BINS_CHG = Calc.loc[:, f'Day Chg Abs 0-500':f'Day Chg Abs 6500-inf'].columns.tolist()
BINS_CHG

['Day Chg Abs 0-500',
 'Day Chg Abs 500-1000',
 'Day Chg Abs 1000-1500',
 'Day Chg Abs 1500-2000',
 'Day Chg Abs 2000-2500',
 'Day Chg Abs 2500-3000',
 'Day Chg Abs 3000-3500',
 'Day Chg Abs 3500-4000',
 'Day Chg Abs 4000-4500',
 'Day Chg Abs 4500-5000',
 'Day Chg Abs 5000-5500',
 'Day Chg Abs 5500-6500',
 'Day Chg Abs 6500-inf']

In [7]:
BINS_HL = Calc.loc[:, f'Day HL 500-1000':f'Day HL 8000-inf'].columns.tolist()
BINS_HL

['Day HL 500-1000',
 'Day HL 1000-1500',
 'Day HL 1500-2000',
 'Day HL 2000-2500',
 'Day HL 2500-3000',
 'Day HL 3000-3500',
 'Day HL 3500-4000',
 'Day HL 4000-4500',
 'Day HL 4500-5000',
 'Day HL 5000-5500',
 'Day HL 5500-6500',
 'Day HL 6500-7000',
 'Day HL 7000-7500',
 'Day HL 7500-8000',
 'Day HL 8000-inf']

In [ ]:
def STATS(Calc, bins):
    # ================ Helpers ================ #
    def _from_clock(x):  return x.hour + x.minute/60
    def   _to_clock(x):  return round(x//1 + x%1 * 60/100, 2)

    # ================ Main ================ #
    pipe = []
    for x in bins: 
        Df1 = Calc[Calc[x]==1] #.reset_index(drop=1)

        for y in ['High','Low']:
            Df2 = Df1[Df1[f'ID {y} Rnk']==1] 

            pipe.append({ 'Bins':x, 'Peak or Valley?':y,
                'Count':          Df2['Time'].count(),
                'Avg':  _to_clock(Df2['Time'].apply(_from_clock).mean()),         
                'Max':  _to_clock(Df2['Time'].apply(_from_clock).max()),          
                'Q3':   _to_clock(Df2['Time'].apply(_from_clock).quantile(.75)),  
                'Med':  _to_clock(Df2['Time'].apply(_from_clock).median()),       
                'Q1':   _to_clock(Df2['Time'].apply(_from_clock).quantile(.25)),  
                'Min':  _to_clock(Df2['Time'].apply(_from_clock).min()),          
            })
        pass
    pass
    return pd.DataFrame(pipe)

In [ ]:
Sts1 = STATS(Calc, bins=BINS_CHG)
Sts1

,Bins,Peak or Valley?,Count,Avg,Max,Q3,Med,Q1,Min
0,Day Chg Abs 0-500,High,317,12.22,18.15,14.15,11.45,10.15,9.00
1,Day Chg Abs 0-500,Low,315,11.49,18.15,13.15,11.15,10.15,9.00
2,Day Chg Abs 500-1000,High,294,12.34,18.15,15.15,11.45,10.00,9.00
3,Day Chg Abs 500-1000,Low,293,12.21,18.15,14.30,11.45,10.15,9.00
4,Day Chg Abs 1000-1500,High,226,12.18,18.15,15.00,11.45,9.15,9.00
5,Day Chg Abs 1000-1500,Low,223,12.44,18.15,15.30,12.00,10.00,9.00
6,Day Chg Abs 1500-2000,High,164,13.04,18.15,16.30,12.22,10.00,9.00
7,Day Chg Abs 1500-2000,Low,161,12.44,18.15,16.00,12.00,9.15,9.00
8,Day Chg Abs 2000-2500,High,104,12.57,18.15,16.15,12.22,9.11,9.00
9,Day Chg Abs 2000-2500,Low,102,12.23,18.15,15.34,11.15,9.00,9.00


In [9]:
Sts2 = STATS(Calc, bins=BINS_HL)
Sts2

,Bins,Peak or Valley?,Count,Avg,Max,Q3,Med,Q1,Min
0,Day HL 500-1000,High,34,12.01,18.15,13.45,11.22,10.19,9.00
1,Day HL 500-1000,Low,29,11.21,18.15,12.00,10.30,9.30,9.00
2,Day HL 1000-1500,High,181,12.27,18.15,15.15,11.30,9.30,9.00
3,Day HL 1000-1500,Low,177,11.39,18.15,13.15,11.00,9.30,9.00
4,Day HL 1500-2000,High,271,12.15,18.15,14.30,11.00,9.30,9.00
5,Day HL 1500-2000,Low,269,12.14,18.15,14.00,11.30,10.15,9.00
6,Day HL 2000-2500,High,244,12.25,18.15,14.49,11.45,10.00,9.00
7,Day HL 2000-2500,Low,246,12.39,18.15,15.15,11.38,10.00,9.00
8,Day HL 2500-3000,High,192,13.08,18.15,16.00,13.15,10.00,9.00
9,Day HL 2500-3000,Low,193,12.20,18.15,15.15,11.30,9.30,9.00


# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']    = Calc_by['Open']  .transform('first')
    Calc['Day High']    = Calc_by['High']  .transform('max')
    Calc['Day Low']     = Calc_by['Low']   .transform('min')
    Calc['Day Close']   = Calc_by['Close'] .transform('last')

    Calc['Day HL']      = Calc['Day High']  - Calc['Day Low']
    Calc['Day Chg']     = Calc['Day Close'] - Calc['Day Open']
    Calc['Day Chg Abs'] = Calc['Day Chg'].abs()

    Calc['ID High Rnk'] = Calc_by['High'].rank(method='dense', ascending=0)
    Calc['ID Low Rnk']  = Calc_by['Low'].rank(method='dense', ascending=1)


    WINS_CHG = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6500,                   inf]
    WINS_HL  = [   500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6500, 7000, 7500, 8000, inf]

    for (a,b) in [x for x in pd.Series(WINS_CHG).rolling(2) if (len(x)==2)]:  
        Calc[f'Day Chg Abs {_int(a)}-{_int(b)}'] = ((a <= Calc['Day Chg Abs']) & (Calc['Day Chg Abs'] < b))  .astype(int)
 
    for (a,b) in [x for x in pd.Series(WINS_HL).rolling(2)  if (len(x)==2)]:       
        Calc[f'Day HL {_int(a)}-{_int(b)}'] = ((a <= Calc['Day HL']) & (Calc['Day HL'] < b))  .astype(int)
 
    return Calc



def STATS(Calc, bins):
    # ================ Helpers ================ #
    def _from_clock(x):  return x.hour + x.minute/60
    def   _to_clock(x):  return round(x//1 + x%1 * 60/100, 2)

    # ================ Main ================ #
    pipe = []
    for x in bins: 
        Df1 = Calc[Calc[x]==1] #.reset_index(drop=1)

        for y in ['High','Low']:
            Df2 = Df1[Df1[f'ID {y} Rnk']==1] 

            pipe.append({ 'Bins':x, 'Peak or Valley?':y,
                'Count':          Df2['Time'].count(),
                'Avg':  _to_clock(Df2['Time'].apply(_from_clock).mean()),         
                'Max':  _to_clock(Df2['Time'].apply(_from_clock).max()),          
                'Q3':   _to_clock(Df2['Time'].apply(_from_clock).quantile(.75)),  
                'Med':  _to_clock(Df2['Time'].apply(_from_clock).median()),       
                'Q1':   _to_clock(Df2['Time'].apply(_from_clock).quantile(.25)),  
                'Min':  _to_clock(Df2['Time'].apply(_from_clock).min()),          
            })
        pass
    pass
    return pd.DataFrame(pipe)